# Running the workflow for Robust-Rail-NL

In [ ]:
import subprocess
import json
import time

### Pipeline for creating scenarios from a configuration file

In [ ]:
# TODO:
# - configurations for scenario generation
#     - give gateway
#     - give service
#     - give in/outstanding

In [ ]:
def create_scenario_config(scenario_number):
    config_content = {
        "location": "kleineBinckhorst",
        "start_time": 0,
        "end_time": 3600,
        "use_default_material": True,
        "trains_given": False,
        "perform_servicing": False,
        "number_of_trains": 6,
        "train_unit_distribution": {
            "train_unit_types": ["SLT-6", "SLT-4", "SNG-3"],
            "type_ratio": 0.2,
            "units_per_composition": [1, 2],
            "matching_complexity": 0.2
        }
    }

    with open(f"./scenario_config_{scenario_number}.json", 'w') as config_file:
        json.dump(config_content, config_file, indent=4)


def call_generator(scenario_number, timeout=60):
    create_scenario_config(scenario_number)
    try:
        result = subprocess.run(['python', '/workspace/robust-rail-generator/src/main.py', "-c", f"./scenario_config_{scenario_number}.json", "-p", ".", "-s", f"./scenario_{scenario_number}.json"], timeout=timeout, stdout=subprocess.PIPE)
        if result.returncode != 0:
            print(f"Solver failed with return code {result.returncode}.")
        print(result.stdout.decode())
    except subprocess.TimeoutExpired:
        print(f"Generator timed out after {timeout} seconds.")
call_generator(6)

### Pipeline for getting different plans from the solver for a given scenario

In [ ]:
# Method to run the solver for a given plan number. 
# It updates the configuration file with the scenario and plan paths, then executes the solver using the updated configuration in subprocess bounded by the timeout.
def run_solver(plan_number, scenario_number, timeout, print_output=False):
    with open("./config_solver.yaml", "r") as f:
        config = f.readlines()
        for i, line in enumerate(config):
            if line.split(": ")[0] == "ScenarioPath":
                config[i] = f'ScenarioPath: "/workspace/scenario-planning-inputs/Scenario_settings/Kleine_Binckhorst_landmarks/scenario_solver_{scenario_number}.json"\n'
            elif line.split(": ")[0] == "PlanPath":
                config[i] = f'PlanPath: "/workspace/scenario-planning-inputs/Scenario_settings/Kleine_Binckhorst_landmarks/scenario_{scenario_number}_plan_{plan_number}.json"\n'
        with open("./config_solver.yaml", "w") as newf:
            newf.writelines(config)


    try:
        print("**Now running the solver for plan number:", plan_number, "***")
        start_time = time.time()
        result = subprocess.run(['dotnet', 'run', '--project', '/workspace/robust-rail-solver/ServiceSiteScheduling/HIP.csproj', '--config=./config_solver.yaml'], timeout=timeout, stdout=subprocess.PIPE)
        if result.returncode != 0:
            print(f"Solver failed with return code {result.returncode}.")
        else:
            print(f"Solver completed in {time.time() - start_time:.2f} seconds.")
        if print_output:
            print("---------- Output: -----------")
            print(result.stdout.decode())
            print("------------------------------")
    except subprocess.TimeoutExpired:
        print(f"Solver timed out after {timeout} seconds.")

In [ ]:
current_timeout = 600
scenarios = [
    # "1",
    # "2",
    # "3",
    # "4",
    # "5",
    # "5a",
    "6"
]
for scenario in scenarios:
    print(f"Running scenario {scenario} with timeout {current_timeout} seconds.")
    for plan_number in range(3,11):
        run_solver(plan_number, scenario, current_timeout)
        print(f"Finished running plan {plan_number} for scenario {scenario}.")
    print("")